<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


Dask DataFrames
===============

Dask dataframes are like pandas dataframes, just bigger.

<img src="https://docs.dask.org/en/stable/_images/dask-dataframe.svg"
     align="right"
     width="30%"
     alt="Dask DataFrame is composed of pandas DataFrames"/>



## Ask for machines

In [1]:
import coiled

cluster = coiled.Cluster(
    n_workers=20,
    region="us-east-2",  # start workers close to data to minimize costs
)

client = cluster.get_client()

╭───────────────────────────────────────── Coiled Cluster ─────────────────────────────────────────╮
│            ]8;id=526549;https://cloud.coiled.io/clusters/210764/overview?account=dask-engineering\https://cloud.coiled.io/clusters/210764/overview?account=dask-engineering]8;;\             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
╭─────────────────── Overview ───────────────────╮╭──────────────── Configuration ─────────────────╮
│                                                ││                                                │
│ Cluster Name: dask-engineering-b9e8c218-c      ││ Region: us-east-2                              │
│                                                ││                                                │
│ Scheduler Status: started                      ││ Scheduler Instance Type: m6i.xlarge            │
│                                                ││                                                │
│ Dashboard Address:                             ││ Worker Instance Type(s): m6i.xlarge (20)       │
│ ]8;id=649502;https://cluster-pqpov.dask.host?token=rpOUHP9Wt0HSA_Gp\https://cluster-pqpov.dask.host?token=rpOUHP9W]8;;\ ││                                                │
│ ]8;id=649502;https://cluster-pqpov.dask.host?token=rpOUHP9Wt0HSA_Gp\t0HSA_Gp]8;;\                                       ││ Workers Requested: 20                          │
│                                                ││                                                │
╰────────────────────────────────────────────────╯╰────────────────────────────────────────────────╯
╭─────────────────────────────────── (2023/05/09 13:02:33 CEST) ───────────────────────────────────╮
│                                                                                                  │
│                    Most of your workers have arrived. Cluster ready for use.                     │
│                                                                                                  │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

## Ingest Uber/Lyft Data


The NYC Taxi dataset is a timeless classic.  

Interestingly there is a new variant.  The NYC Taxi and Livery Commission requires data from all ride-share services in the city of New York.  This includes private limosine services, van services, and a new category "High Volume For Hire Vehicle" services, those that dispatch 10,000 rides per day or more.  This is a special category defined for Uber and Lyft.  

In [ ]:
import dask
import pandas
import dask.dataframe as dd
import pandas as pd

dask.config.set({"dataframe.convert-string": True})  # use PyArrow strings by default

# df = pd.read_parquet(  # this would work if we had enough memory
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={"anon": True},
)
df.head()

Play time
---------

We actually don't know what to expect from this dataset.  No one in our team has spent much time inspecting it.  We'd like to solicit help from you, new Dask user, to uncover some interesting insights.  

Care to explore and report your findings?

In [ ]:
df = df.persist()

df.columns

In [ ]:
df.head()

## Basic statistics

In [ ]:
total = df[["base_passenger_fare", "driver_pay", "tips", "trip_miles"]].sum()
average = df[["base_passenger_fare", "driver_pay", "tips", "trip_miles"]].mean()

In [ ]:
total

In [ ]:
average

In [ ]:
total, average = dask.compute(total, average)

## Tipping Practices

In [ ]:
# How often do New Yorkers tip?

(df.tips != 0).mean().compute()

## Broken down by carrier

In [ ]:
# Uber / Lyft / Via / ... different carriers
df.hvfhs_license_num.value_counts().compute()

In [ ]:
df["tipped"] = df.tips != 0

df.groupby("hvfhs_license_num").tipped.mean().compute()

## Dask TV

We use this in conference events just to make the dashboard go and bring in a crowd.  Colloquially we call this "Dask TV".  Enjoy!

In [ ]:
import dask
dask.config.set({"dataframe.convert-string": True})  # use PyArrow strings by default

while True:
    client.restart()

    df = dd.read_parquet(
        "s3://coiled-datasets/uber-lyft-tlc/",
        storage_options={"anon": True},
    ).persist()

    for _ in range(10):
        df["tipped"] = df.tips != 0

        df.groupby("hvfhs_license_num").tipped.mean().compute()